In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


def clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared and memory freed.")

In [ ]:
import json
import os
import re
from datasets import Dataset, DatasetDict
from transformers import (
    T5Tokenizer,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
import nltk
from typing import List, Tuple
from nltk.tokenize import sent_tokenize
from datasets import Dataset, concatenate_datasets
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
import torch
from sklearn.model_selection import train_test_split

2025-04-06 01:25:46.548464: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 01:25:46.565218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743902746.585703    7321 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743902746.591963    7321 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 01:25:46.613267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Training code

In [4]:
!pip install accelerate -U

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install evaluate nltk rouge_score bert_score transformers[torch]

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
## data preparation and making decisions for additional tokens

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(
    tokenizer.tokenize(
        "brick:Natural_Gas rdfs:label ?label SELECT WHERE brick:Air_Flow_Sensor"
    )
)
# Output: ['brick', ':', 'Natural', '_', 'Gas', 'rdfs', ':', 'label', '?', 'label']

['▁brick', ':', 'N', 'atura', 'l', '_', 'G', 'a', 's', '▁', 'r', 'd', 'f', 's', ':', 'l', 'abel', '▁', '?', 'l', 'abel', '▁', 'SEL', 'ECT', '▁W', 'HER', 'E', '▁brick', ':', 'Air', '_', 'Flow', '_', 'S', 'en', 's', 'or']


In [ ]:
import json

# Load the first JSON file
with open("updated_bldg_question_pairs_entities.json", "r") as f1:
    data1 = json.load(f1)

# Load the second JSON file
with open("updated_combined_output_with_entity.json", "r") as f2:
    data2 = json.load(f2)


# Function to filter and extract only the required keys
def filter_entry(entry):
    return {
        "question": entry["question"],
        "entity": entry["entity"],
        "sparql": entry["sparql"],
    }


# Process both datasets and combine them
merged_data = []

# Filter entries from file1.json
for entry in data1:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
for entry in data2:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Save the merged dataset to a new JSON file
with open("merged_output1.json", "w") as f:
    json.dump(merged_data, f, indent=4)

print(
    "Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'."
)

Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.


In [ ]:
import json

# Load the first JSON file
with open("abacws_bldg_question_pairs_entities.json", "r") as f1:
    data1 = json.load(f1)

# Load the second JSON file
with open("abacws_bldg_timeseries_question_pairs_entities.json", "r") as f2:
    data2 = json.load(f2)

#  # Load the second JSON file
# with open("updated_bldg_question_pairs_entities.json", "r") as f3:
#     data3 = json.load(f3)


# Function to filter and extract only the required keys
def filter_entry(entry):
    return {
        "question": entry["question"],
        "entity": entry["entity"],
        "sparql": entry["sparql"],
    }


# Process both datasets and combine them
merged_data = []

# Filter entries from file1.json
for entry in data1:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
for entry in data2:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
# for entry in data3:
#     filtered_entry = filter_entry(entry)
#     merged_data.append(filtered_entry)

# Save the merged dataset to a new JSON file
with open("merged_output2.json", "w") as f:
    json.dump(merged_data, f, indent=4)

print(
    "Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'."
)

Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.


In [ ]:
import json

# Load the first JSON file
with open("merged_output2.json", "r") as f1:
    data1 = json.load(f1)
  
# Load the second JSON file
with open("merged_output3.json", "r") as f2:
    data2 = json.load(f2)


# Function to filter and extract only the required keys
def filter_entry(entry):
    return {
        "question": entry["question"],
        "entity": entry["entity"],
        "sparql": entry["sparql"],
    }


# Process both datasets and combine them
merged_data = []

# Filter entries from file1.json
for entry in data1:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Filter entries from file2.json
for entry in data2:
    filtered_entry = filter_entry(entry)
    merged_data.append(filtered_entry)

# Save the merged dataset to a new JSON file
with open("training_data.json", "w") as f:
    json.dump(merged_data, f, indent=4)

print(
    "Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'."
)

Files merged successfully into 'merged_output.json' with keys: 'question', 'entity', and 'sparql'.


In [ ]:
import json


# Function to read names from a file (one per line)
def load_names_from_file(filename):
    with open(filename, "r") as f:
        # Strip whitespace and filter out empty lines
        names = [line.strip() for line in f if line.strip()]
    return names


# Load sensors and zones from files
sensors = load_names_from_file("brick_sensors.txt")
zones = load_names_from_file("bldg_zones.txt")

# Add 'brick:' prefix to sensors and 'bldg:' prefix to zones if not already present
sensors = [f"brick:{s}" if not s.startswith("brick:") else s for s in sensors]
zones = [f"bldg:{z}" if not z.startswith("bldg:") else z for z in zones]


# Function to generate SPARQL queries and dataset entries
def generate_sparql_dataset(sensors_list, zones_list):
    dataset = []

    # Base entity template (e.g., "bldg:<Zone> \n brick:<SensorName>")
    def get_entity(sensor, zone):
        sensor_name = sensor.split(":")[1]  # Extract sensor name after "brick:"
        zone_name = zone.split(":")[1]  # Extract zone name after "bldg:"
        return f"bldg:{zone_name} \n brick:{sensor_name}"  # Newline-separated format

    # Loop over each sensor and each zone
    for sensor in sensors_list:
        for zone in zones_list:
            selected_entity = get_entity(sensor, zone)

            # Create the question string with sensor and zone details
            question = (
                f"Tell me the name or label of the {sensor.split(':')[1].replace('_', ' ').lower()} "
                f"in the {zone.split(':')[1].replace('-', ' ').replace('_', ' ')}."
            )

            # Create the corresponding SPARQL query
            sparql_query = f"SELECT ?label WHERE {{ ?sensor a {sensor} ; brick:hasLocation {zone} ; rdfs:label ?label . }}"

            # Create an entry and append it to the dataset
            entry = {
                "question": question,
                "entity": selected_entity,
                "sparql": sparql_query,
            }
            dataset.append(entry)

    return dataset


# Generate the dataset (will produce 5000 entries if there are 100 sensors and 50 zones)
dataset = generate_sparql_dataset(sensors, zones)

# Save to JSON file
with open("merged_output3.json", "w") as f:
    json.dump(dataset, f, indent=4)

print("Dataset generated and saved to 'sparql_dataset1.json'.")

Dataset generated and saved to 'sparql_dataset1.json'.


In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


def clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared and memory freed.")


import os

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import gc
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import evaluate
import torch
import nltk
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq

# Set CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
nltk.download("punkt", quiet=True)

# 1. LOAD AND PREPARE DATA
DATA_FILE = "updated_combined_output_with_entity.json"
with open(DATA_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} records from {DATA_FILE}")
print("Example record:", data[0])

# Create multi-task training pairs
inputs, targets = [], []
for record in data:
    question = record.get("question", "")
    entity = record.get("entity", "")
    sparql = record.get("sparql", "")
    # response = record.get("sparql_response", "")
    # explanation = record.get("explanation", "")

    # Task 1: NL to SPARQL
    if question and entity and sparql:
        inputs.append(f"task: generate_sparql\ninput: {question}\nentity{entity}")
        targets.append(sparql)

    # # Task 2: Summarize response
    # if question and response and explanation:
    #     inputs.append(f"task: summarize_response\nquestion: {question}\nresponse: {response}")
    #     targets.append(explanation)

print(f"Generated {len(inputs)} total training pairs from {len(data)} records.")
train_inputs, val_inputs, train_targets, val_targets = train_test_split(
    inputs, targets, test_size=0.1, random_state=42
)
print(f"Train size: {len(train_inputs)} | Validation size: {len(val_inputs)}")

# Save splits
train_data = [
    {"input_text": inp, "target_text": tgt}
    for inp, tgt in zip(train_inputs, train_targets)
]
val_data = [
    {"input_text": inp, "target_text": tgt} for inp, tgt in zip(val_inputs, val_targets)
]
with open("train_data_2April.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)
with open("val_data_2April.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)
print("Saved train_data_2April.json and val_data_2April.json!")

# Build datasets
raw_datasets = DatasetDict(
    {
        "train": Dataset.from_dict(
            {"input_text": train_inputs, "target_text": train_targets}
        ),
        "validation": Dataset.from_dict(
            {"input_text": val_inputs, "target_text": val_targets}
        ),
    }
)
print("Train sample:", raw_datasets["train"][3])
print("Validation sample:", raw_datasets["validation"][3])

# 2. LOAD MODEL & TOKENIZER
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Add custom tokens once
# custom_tokens = []
# with open("all_relations_and_classes.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# with open("output_entities.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# num_added_tokens = tokenizer.add_tokens(custom_tokens)
# model.resize_token_embeddings(len(tokenizer))
# print(f"Added {num_added_tokens} new tokens to the tokenizer!")


# 3. PREPROCESSING
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"], max_length=512, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        text_target=examples["target_text"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 4. EVALUATION METRICS
metric_rouge = evaluate.load("rouge")
metric_bleu = evaluate.load("bleu")
metric_meteor = evaluate.load("meteor")
metric_bertscore = evaluate.load("bertscore")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    results = {}
    rouge_result = metric_rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    results.update(rouge_result)
    bleu_result = metric_bleu.compute(
        predictions=decoded_preds, references=[[label] for label in decoded_labels]
    )
    results["bleu"] = bleu_result["bleu"]
    meteor_result = metric_meteor.compute(
        predictions=decoded_preds, references=decoded_labels
    )
    results["meteor"] = meteor_result["meteor"]
    bertscore_result = metric_bertscore.compute(
        predictions=decoded_preds, references=decoded_labels, lang="en"
    )
    results["bertscore_precision"] = np.mean(bertscore_result["precision"])
    results["bertscore_recall"] = np.mean(bertscore_result["recall"])
    results["bertscore_f1"] = np.mean(bertscore_result["f1"])
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    results["gen_len"] = np.mean(prediction_lens)
    return results


# 5. TRAINING ARGUMENTS
training_args = Seq2SeqTrainingArguments(
    output_dir="./training_t5small",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    logging_strategy="epoch",
    logging_dir="./training_t5small",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on GPU
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,  # Enable if GPU supports
    report_to=["tensorboard"],
    warmup_steps=500,
    lr_scheduler_type="cosine",
)

# 6. TRAINER SETUP
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# 7. TRAIN
trainer.train()

Loaded 52599 records from updated_combined_output_with_entity.json
Example record: {'question': 'What is the area of building bldg1?', 'entity': 'brick:area', 'sparql': 'SELECT ?value ?unit WHERE { bldg:bldg1 brick:area ?area . ?area brick:value ?value . ?area brick:hasUnits ?unit . }', 'sparql_response': '[{"value": "9973^^xsd:integer", "unit": "FT_2"}]', 'explanation': 'The area of building bldg1 is 9,973 square feet (FT_2). This means that the total floor space occupied by this building is approximately 9,973 square feet. This measurement is commonly used for real estate and construction purposes to describe the size of a building or space. In this case, the number "9,973" represents the precise area of building bldg1 as obtained from the smart building data created using Brickschema ontology. The "FT_2" notation signifies that the unit of measurement is square feet (foot squared).', 'id': 1}
Generated 101897 total training pairs from 52599 records.
Train size: 91707 | Validation si

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Added 3157 new tokens to the tokenizer!


Map:   0%|          | 0/91707 [00:00<?, ? examples/s]

Map:   0%|          | 0/10190 [00:00<?, ? examples/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Meteor,Bertscore Precision,Bertscore Recall,Bertscore F1,Gen Len
1,1.323800,0.661418,0.244202,0.099510,0.233062,0.233117,0.000193,0.192074,0.855569,0.807940,0.830578,18.784396
2,0.685800,0.587053,0.280404,0.125103,0.269910,0.270053,0.000123,0.278528,0.882677,0.812443,0.845823,18.974092
3,0.626500,0.544086,0.281515,0.134946,0.271180,0.271287,0.000113,0.269733,0.885476,0.812579,0.847177,18.958292
4,0.588100,0.512603,0.281397,0.135671,0.271331,0.271424,0.000109,0.272059,0.885017,0.812548,0.846957,18.962022
5,0.558900,0.488756,0.313850,0.184582,0.304589,0.304690,0.000141,0.302181,0.888935,0.817713,0.851505,18.970167
6,0.536200,0.469700,0.319792,0.195133,0.310231,0.310288,0.000140,0.309091,0.887229,0.818980,0.851352,18.977625
7,0.518000,0.454499,0.325723,0.198744,0.314771,0.314795,0.000173,0.318325,0.885552,0.822105,0.852180,18.990186
8,0.503300,0.442119,0.329566,0.209941,0.320139,0.320037,0.000182,0.322040,0.884684,0.823457,0.852467,18.990285
9,0.491100,0.431673,0.338236,0.223992,0.329603,0.329625,0.000193,0.327153,0.885391,0.824366,0.853273,18.987929
10,0.480900,0.423087,0.347757,0.234342,0.339611,0.339731,0.000193,0.337299,0.886209,0.825573,0.854318,18.986457


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to 

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


def clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared and memory freed.")


import os

os.environ["WANDB_MODE"] = "offline"

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import gc
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import evaluate
import torch
import nltk
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq

# Set CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
nltk.download("punkt", quiet=True)

# 1. LOAD AND PREPARE DATA
DATA_FILE = "updated_combined_output_with_entity.json"
with open(DATA_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)
print(f"Loaded {len(data)} records from {DATA_FILE}")
print("Example record:", data[0])

# Create multi-task training pairs
inputs, targets = [], []
for record in data:
    question = record.get("question", "")
    entity = record.get("entity", "")
    sparql = record.get("sparql", "")
    response = record.get("sparql_response", "")
    explanation = record.get("explanation", "")

    # Task 1: NL to SPARQL
    if question and entity and sparql:
        inputs.append(f"task: generate_sparql\ninput: {question}\nentity{entity}")
        targets.append(sparql)

    # Task 2: Summarize response
    if question and response and explanation:
        inputs.append(
            f"task: summarize_response\nquestion: {question}\nresponse: {response}"
        )
        targets.append(explanation)

print(f"Generated {len(inputs)} total training pairs from {len(data)} records.")
train_inputs, val_inputs, train_targets, val_targets = train_test_split(
    inputs, targets, test_size=0.1, random_state=40
)
print(f"Train size: {len(train_inputs)} | Validation size: {len(val_inputs)}")

# Save splits
train_data = [
    {"input_text": inp, "target_text": tgt}
    for inp, tgt in zip(train_inputs, train_targets)
]
val_data = [
    {"input_text": inp, "target_text": tgt} for inp, tgt in zip(val_inputs, val_targets)
]
with open("train_data_2April.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)
with open("val_data_2April.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)
print("Saved train_data_2April.json and val_data_2April.json!")

# Build datasets
raw_datasets = DatasetDict(
    {
        "train": Dataset.from_dict(
            {"input_text": train_inputs, "target_text": train_targets}
        ),
        "validation": Dataset.from_dict(
            {"input_text": val_inputs, "target_text": val_targets}
        ),
    }
)
print("Train sample:", raw_datasets["train"][3])
print("Validation sample:", raw_datasets["validation"][3])

# 2. LOAD MODEL & TOKENIZER
model_name = "./training_t5smallv2/checkpoint-343920"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Add custom tokens once
# custom_tokens = []
# with open("all_relations_and_classes.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# with open("output_entities.txt", "r", encoding="utf-8") as f:
#     custom_tokens.extend([line.strip() for line in f.readlines()])
# num_added_tokens = tokenizer.add_tokens(custom_tokens)
# model.resize_token_embeddings(len(tokenizer))
# print(f"Added {num_added_tokens} new tokens to the tokenizer!")


# 3. PREPROCESSING
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"], max_length=512, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        text_target=examples["target_text"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 4. EVALUATION METRICS
metric_rouge = evaluate.load("rouge")
metric_bleu = evaluate.load("bleu")
metric_meteor = evaluate.load("meteor")
metric_bertscore = evaluate.load("bertscore")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    results = {}
    rouge_result = metric_rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    results.update(rouge_result)
    bleu_result = metric_bleu.compute(
        predictions=decoded_preds, references=[[label] for label in decoded_labels]
    )
    results["bleu"] = bleu_result["bleu"]
    meteor_result = metric_meteor.compute(
        predictions=decoded_preds, references=decoded_labels
    )
    results["meteor"] = meteor_result["meteor"]
    bertscore_result = metric_bertscore.compute(
        predictions=decoded_preds, references=decoded_labels, lang="en"
    )
    results["bertscore_precision"] = np.mean(bertscore_result["precision"])
    results["bertscore_recall"] = np.mean(bertscore_result["recall"])
    results["bertscore_f1"] = np.mean(bertscore_result["f1"])
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    results["gen_len"] = np.mean(prediction_lens)
    return results


# 5. TRAINING ARGUMENTS
training_args = Seq2SeqTrainingArguments(
    output_dir="./training_t5small",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    logging_strategy="epoch",
    logging_dir="./training_t5small",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on GPU
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,  # Enable if GPU supports
    report_to=["tensorboard"],
    warmup_steps=500,
    lr_scheduler_type="cosine",
)

# 6. TRAINER SETUP
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# 7. TRAIN
trainer.train()

Loaded 52599 records from updated_combined_output_with_entity.json
Example record: {'question': 'What is the area of building bldg1?', 'entity': 'brick:area', 'sparql': 'SELECT ?value ?unit WHERE { bldg:bldg1 brick:area ?area . ?area brick:value ?value . ?area brick:hasUnits ?unit . }', 'sparql_response': '[{"value": "9973^^xsd:integer", "unit": "FT_2"}]', 'explanation': 'The area of building bldg1 is 9,973 square feet (FT_2). This means that the total floor space occupied by this building is approximately 9,973 square feet. This measurement is commonly used for real estate and construction purposes to describe the size of a building or space. In this case, the number "9,973" represents the precise area of building bldg1 as obtained from the smart building data created using Brickschema ontology. The "FT_2" notation signifies that the unit of measurement is square feet (foot squared).', 'id': 1}
Generated 101897 total training pairs from 52599 records.
Train size: 91707 | Validation si

Map:   0%|          | 0/91707 [00:00<?, ? examples/s]

Map:   0%|          | 0/10190 [00:00<?, ? examples/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Meteor,Bertscore Precision,Bertscore Recall,Bertscore F1,Gen Len
1,0.295700,0.248385,0.504155,0.433181,0.496060,0.496050,0.005084,0.480016,0.934897,0.888933,0.910661,18.019725
2,0.294300,0.248268,0.507756,0.438752,0.499885,0.499957,0.005157,0.482675,0.935279,0.889683,0.911232,18.021295
3,0.293000,0.248041,0.511299,0.443744,0.503168,0.503253,0.005315,0.484156,0.936100,0.890650,0.912128,18.020510


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to 

KeyboardInterrupt: 

### Evaluation code

     |████████████████████████████████| 61 kB 7.0 MB/s eta 0:00:011
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import json
import torch
import numpy as np
from transformers import T5ForConditionalGeneration
import os
import torch
import numpy as np
from transformers import (
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback,
    T5Tokenizer,
    DataCollatorForSeq2Seq,
)
import gc
import json
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import evaluate
import nltk
import numpy as np

# -----------------------------------------------------------------------------
# 1. LOAD THE FINE-TUNED MODEL AND TOKENIZER
# -----------------------------------------------------------------------------
model_name = "./training_t5small/checkpoint-343920"  # Use your latest checkpoint
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on {device}")

# -----------------------------------------------------------------------------
# 2. LOAD EVALUATION DATA
# -----------------------------------------------------------------------------
# Option 1: Load from validation split saved during training
val_data_file = "val_data_2April.json"
with open(val_data_file, "r", encoding="utf-8") as f:
    val_data = json.load(f)
print(f"Loaded {len(val_data)} validation records from {val_data_file}")

# Separate tasks
sparql_inputs = []
sparql_targets = []
summarization_inputs = []
summarization_targets = []

for record in val_data:
    input_text = record["input_text"]
    target_text = record["target_text"]

    if "task: generate_sparql" in input_text:
        sparql_inputs.append(input_text)
        sparql_targets.append(target_text)
    elif "task: summarize_response" in input_text:
        summarization_inputs.append(input_text)
        summarization_targets.append(target_text)

print(f"SPARQL task: {len(sparql_inputs)} examples")
print(f"Summarization task: {len(summarization_inputs)} examples")


# -----------------------------------------------------------------------------
# 3. DEFINE HELPER FUNCTIONS
# -----------------------------------------------------------------------------
def generate_predictions(input_texts, max_length=512):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text in input_texts:
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=512,
            ).to(device)
            output_ids = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                num_beams=4,  # Beam search for better quality
                early_stopping=True,
            )
            pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            predictions.append(pred.strip())
    return predictions


# -----------------------------------------------------------------------------
# 4. LOAD EVALUATION METRICS
# -----------------------------------------------------------------------------
metric_rouge = evaluate.load("rouge")
metric_bleu = evaluate.load("bleu")
metric_meteor = evaluate.load("meteor")
metric_bertscore = evaluate.load("bertscore")


def compute_metrics(predictions, references, task_name=""):
    results = {}
    # ROUGE
    rouge_result = metric_rouge.compute(
        predictions=predictions, references=references, use_stemmer=True
    )
    results.update({f"{task_name}_rouge_{k}": v for k, v in rouge_result.items()})

    # BLEU
    bleu_result = metric_bleu.compute(
        predictions=predictions, references=[[ref] for ref in references]
    )
    results[f"{task_name}_bleu"] = bleu_result["bleu"]

    # METEOR
    meteor_result = metric_meteor.compute(
        predictions=predictions, references=references
    )
    results[f"{task_name}_meteor"] = meteor_result["meteor"]

    # BERTScore
    bertscore_result = metric_bertscore.compute(
        predictions=predictions, references=references, lang="en"
    )
    results[f"{task_name}_bertscore_precision"] = np.mean(bertscore_result["precision"])
    results[f"{task_name}_bertscore_recall"] = np.mean(bertscore_result["recall"])
    results[f"{task_name}_bertscore_f1"] = np.mean(bertscore_result["f1"])

    # Average generation length
    prediction_lens = [len(pred.split()) for pred in predictions]
    results[f"{task_name}_gen_len"] = np.mean(prediction_lens)

    return results


# -----------------------------------------------------------------------------
# 5. EVALUATE SPARQL GENERATION TASK
# -----------------------------------------------------------------------------
print("\n=== Evaluating SPARQL Generation Task ===")
sparql_predictions = generate_predictions(sparql_inputs, max_length=128)

# Print a few examples
for i, (inp, pred, tgt) in enumerate(
    zip(sparql_inputs[:3], sparql_predictions[:3], sparql_targets[:3])
):
    print(f"Example {i+1}:")
    print("Input:     ", inp)
    print("Prediction:", pred)
    print("Target:    ", tgt)
    print("-" * 50)

# Compute metrics
sparql_metrics = compute_metrics(sparql_predictions, sparql_targets, task_name="sparql")
print("\nSPARQL Generation Metrics:")
for k, v in sparql_metrics.items():
    print(f"{k}: {v:.4f}")

# -----------------------------------------------------------------------------
# 6. EVALUATE SUMMARIZATION TASK
# -----------------------------------------------------------------------------
print("\n=== Evaluating Summarization Task ===")
summarization_predictions = generate_predictions(summarization_inputs, max_length=256)

# Print a few examples
for i, (inp, pred, tgt) in enumerate(
    zip(
        summarization_inputs[:3],
        summarization_predictions[:3],
        summarization_targets[:3],
    )
):
    print(f"Example {i+1}:")
    print("Input:     ", inp)
    print("Prediction:", pred)
    print("Target:    ", tgt)
    print("-" * 50)

# Compute metrics
summarization_metrics = compute_metrics(
    summarization_predictions, summarization_targets, task_name="summarization"
)
print("\nSummarization Metrics:")
for k, v in summarization_metrics.items():
    print(f"{k}: {v:.4f}")

# -----------------------------------------------------------------------------
# 7. CLEAR CUDA CACHE (Optional)
# -----------------------------------------------------------------------------
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDA cache cleared.")

Model loaded on cuda
Loaded 10190 validation records from val_data_2April.json
SPARQL task: 5187 examples
Summarization task: 5003 examples


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



=== Evaluating SPARQL Generation Task ===
Example 1:
Input:      task: generate_sparql
input: What types is the Occupied Heating Mode Status?
entitybrick:Occupied_Heating_Mode_Status
Prediction: SELECT?definition WHERE?quantity skos:definition?definition.
Target:     SELECT ?type WHERE { brick:Occupied_Heating_Mode_Status a ?type . }
--------------------------------------------------
Example 2:
Input:      task: generate_sparql
input: Provide the definition for Air Flow Setpoint.
entitybrick:Air_Flow_Setpoint
Prediction: SELECT?definition WHERE brick:Fan skos:definition?definition.
Target:     SELECT ?definition WHERE { brick:Air_Flow_Setpoint skos:definition ?definition . }
--------------------------------------------------
Example 3:
Input:      task: generate_sparql
input: Can you provide the definition for the Return Air Humidity Sensor?
entitybrick:Return_Air_Humidity_Sensor
Prediction: SELECT?definition WHERE?parent skos:definition?definition.
Target:     SELECT ?definition WHER

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



SPARQL Generation Metrics:
sparql_rouge_rouge1: 0.6714
sparql_rouge_rouge2: 0.5177
sparql_rouge_rougeL: 0.6709
sparql_rouge_rougeLsum: 0.6709
sparql_bleu: 0.3783
sparql_meteor: 0.6631
sparql_bertscore_precision: 0.9139
sparql_bertscore_recall: 0.8740
sparql_bertscore_f1: 0.8934
sparql_gen_len: 3.8359

=== Evaluating Summarization Task ===
Example 1:
Input:      task: summarize_response
question: What is the quantity linked to the Return Air Enthalpy Sensor?
response: [{"quantity": "Enthalpy"}]
Prediction: The Return Air Enthalpy Sensor in this smart building system measures the enthalpy of the air being returned to the HVAC (Heating, Ventilation, and Air Conditioning) system. Enthalpy is a thermodynamic property that describes the total energy content of a substance (in this case, air) in a given volume of air. In simpler terms, the Return Air Enthalpy Sensor measures the enthalpy of the air being returned to the heating, ventilation, and air conditioning (HVAC) system after it has ci

### test on new data 

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# -----------------------------------------------------------------------------
# 1. LOAD THE TRAINED MODEL AND TOKENIZER
# -----------------------------------------------------------------------------
model_name = "./training_t5small/checkpoint-57320"  # Update with your checkpoint path
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on {device}")


# -----------------------------------------------------------------------------
# 2. HELPER FUNCTION TO GENERATE RESPONSES
# -----------------------------------------------------------------------------
def generate_response(input_text, max_length=256):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512,
        ).to(device)
        output_ids = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            num_beams=4,  # Beam search for better quality
            early_stopping=True,
        )
        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response.strip()


# -----------------------------------------------------------------------------
# 3. INTERACTIVE TESTING LOOP
# -----------------------------------------------------------------------------
def test_model():
    print("\n=== Interactive Model Testing ===")
    print("Enter inputs for SPARQL generation or summarization tasks.")
    print(
        "For SPARQL: Provide a natural language question (e.g., 'What is the capital of France?')"
    )
    print(
        "For Summarization: Provide a question and response (e.g., 'What is the weather like? [Weather data]')"
    )
    print("Type 'exit' to quit.\n")

    while True:
        # Get user input
        task_choice = input("Choose task (1 for SPARQL, 2 for Summarization): ").strip()

        if task_choice.lower() == "exit":
            break

        if task_choice not in ["1", "2"]:
            print("Invalid choice. Please enter 1 for SPARQL or 2 for Summarization.")
            continue

        # Task 1: SPARQL Generation
        if task_choice == "1":
            question = input("Enter your question: ").strip()
            if not question:
                print("Please provide a question.")
                continue

            # Add entity if provided (optional)
            entity = input("Enter entity (optional, press Enter to skip): ").strip()
            input_text = f"task: generate_sparql\ninput: {question}"
            if entity:
                input_text += f"\nentity: {entity}"

            # Generate SPARQL
            sparql_response = generate_response(input_text, max_length=128)
            print("\nGenerated SPARQL:")
            print(sparql_response)
            print("-" * 50)

        # Task 2: Summarization
        elif task_choice == "2":
            question = input("Enter your question: ").strip()
            response = input("Enter the response to summarize: ").strip()
            if not question or not response:
                print("Please provide both a question and a response.")
                continue

            input_text = (
                f"task: summarize_response\nquestion: {question}\nresponse: {response}"
            )

            # Generate summary
            summary_response = generate_response(input_text, max_length=256)
            print("\nGenerated Summary:")
            print(summary_response)
            print("-" * 50)


# -----------------------------------------------------------------------------
# 4. RUN THE TEST
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    test_model()

    # Clear CUDA cache after testing (optional)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared.")

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load your trained model and tokenizer from the saved directory.
model_dir = "./training_t5smallv2/checkpoint-343920"  # Update this path if needed
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def generate_sparql(question, entity):
    """
    Generate a SPARQL query given a natural language question and an entity.
    """
    # Format the input as it was during training.
    input_text = f"task: generate_sparql\ninput: {question}\nentity{entity}"
    input_ids = tokenizer.encode(
        input_text, return_tensors="pt", truncation=True, max_length=512
    ).to(device)

    # Generate output using beam search.
    outputs = model.generate(
        input_ids, max_length=150, num_beams=5, early_stopping=True
    )
    generated_sparql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_sparql


def summarize_response(question, response):
    """
    Generate a summary (explanation) based on the question and response text.
    """
    # Format the input as it was during training.
    input_text = f"task: summarize_response\nquestion: {question}\nresponse: {response}"
    input_ids = tokenizer.encode(
        input_text, return_tensors="pt", truncation=True, max_length=512
    ).to(device)

    # Generate summary output.
    outputs = model.generate(
        input_ids, max_length=256, num_beams=5, early_stopping=True
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


# Example input data
input_data = {
    "question": "Provide the definition for Heating Ventilation Air Conditioning System.",
    "entity": "brick:Heating_Ventilation_Air_Conditioning_System",
    "sparql": "SELECT ?definition WHERE { brick:Heating_Ventilation_Air_Conditioning_System skos:definition ?definition . }",
    "sparql_response": '[{"definition": "The equipment, distribution systems and terminals that provide, either collectively or individually, the processes of heating, ventilating or air conditioning to a building or portion of a building"}]',
    "explanation": (
        "The Heating Ventilation Air Conditioning (HVAC) System is a set of equipment, systems, and components that work together "
        "to control and maintain comfortable conditions within a building. This includes processes such as heating the interior space during "
        "cold weather, cooling it during warm weather, moving air for ventilation, and maintaining the desired indoor air quality. The HVAC "
        "system is crucial for providing a comfortable and healthy environment in any building, be it residential or commercial. It can "
        "consist of various components like air handlers, ducts, vents, thermostats, boilers, furnaces, cooling towers, heat pumps, and more, "
        "depending on the specific needs of the building. The HVAC system is designed to ensure that the temperature, humidity, and air quality "
        "within the building are maintained at optimal levels for the comfort and health of its occupants."
    ),
    "id": 49568,
}

# Get outputs for both tasks
sparql_output = generate_sparql(input_data["question"], input_data["entity"])
summary_output = summarize_response(
    input_data["question"], input_data["sparql_response"]
)

print("Generated SPARQL Query:")
print(sparql_output)
print("\nGenerated Summary:")
print(summary_output)

Generated SPARQL Query:
SELECT?definition WHERE brick:Heating_Ventilation_Air_Conditioning_System skos:definition?definition.

Generated Summary:
A Heating Ventilation Air Conditioning (HVAC) System refers to the collection of equipment, distribution systems, and terminals that provide, collectively or individually, the processes of heating, ventilating, or air conditioning to a specific building or portion of a building. This system is responsible for maintaining a comfortable indoor environment by regulating temperature, humidity, and air quality. The primary function of a HVAC system is to maintain a comfortable temperature for occupants while also ensuring energy efficiency.


In [3]:
def add_prefix(input_file: str, output_file: str):
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            # Remove any trailing newline or whitespace
            line = line.strip()
            if line:
                # Prepend "bldg:" to the line and write it to the output file
                outfile.write(f" - {line}\n")


if __name__ == "__main__":
    add_prefix("bldg_sensors.txt", "bldg_sensors_sufix.txt")
    print("bldg_sensors_dash.txt")

bldg_sensors_dash.txt
